# AccessiTrip AI: Smart Accessible Travel Planner

A 2-hour MVP demo leveraging Databricks, Unity Catalog, Mosaic AI, Delta, RAG, agent tools, MLflow, and Streamlit.  
**Purpose:** Help travelers (with or without accessibility needs) find accessible places to stay and visit in any US city, using Bright Initiative datasets (Airbnb, Booking.com, Google Maps Businesses).

---

## 1. Data Access: Delta Tables & Demo Data

This notebook can use either real Delta tables (from Unity Catalog) or in-memory demo data. Toggle the `USE_DEMO_DATA` flag below.

- **Delta Table Mode:** Reads from Unity Catalog tables (Airbnb, Booking.com, Google Maps Businesses). See `partner_data_quickstart.ipynb` and `nimble-mcp.ipynb` for patterns.
- **Demo Mode:** Uses small in-memory DataFrames for fast prototyping.

In [ ]:
# Toggle this to False to use real Delta tables
USE_DEMO_DATA = True

## 2. RAG: Databricks Vector Search Integration

This section demonstrates how to use Databricks Vector Search for retrieval-augmented generation (RAG) over your Delta tables. (See also: vector_search_fm_api.ipynb)

In [ ]:
from databricks.vector_search.client import VectorSearchClient

# Set up vector search client and index names
CATALOG = "main"
DB = "bright_airbnb"
SOURCE_TABLE_NAME = "listings"
SOURCE_TABLE_FULLNAME = f"{CATALOG}.{DB}.{SOURCE_TABLE_NAME}"
VS_ENDPOINT_NAME = "vs_endpoint"
VS_INDEX_NAME = "accessitrip_vs_index"
VS_INDEX_FULLNAME = f"{CATALOG}.{DB}.{VS_INDEX_NAME}"

vsc = VectorSearchClient()

# Ensure endpoint exists
if vsc.list_endpoints().get('endpoints') is None or not VS_ENDPOINT_NAME in [e.get('name') for e in vsc.list_endpoints().get('endpoints')]:
    vsc.create_endpoint(VS_ENDPOINT_NAME)
vsc.wait_for_endpoint(VS_ENDPOINT_NAME, 600)

# Ensure index exists
if not VS_INDEX_FULLNAME in [i.get("name") for i in vsc.list_indexes(VS_ENDPOINT_NAME).get('vector_indexes', [])]:
    vsc.create_delta_sync_index_and_wait(
        endpoint_name=VS_ENDPOINT_NAME,
        index_name=VS_INDEX_FULLNAME,
        source_table_name=SOURCE_TABLE_FULLNAME,
        pipeline_type="TRIGGERED",
        primary_key="id",
        embedding_source_column="description",
        embedding_model_endpoint_name="databricks-bge-large-en"
    )

index = vsc.get_index(endpoint_name=VS_ENDPOINT_NAME, index_name=VS_INDEX_FULLNAME)

# Example RAG query
rag_results = index.similarity_search(
    columns=["name", "description", "amenities"],
    query_text="wheelchair accessible hotel in Chicago",
    num_results=3
)

# Convert results to DataFrame for downstream use
import pandas as pd
if rag_results and rag_results.get('result') and rag_results['result'].get('data_array'):
    rag_df = pd.DataFrame(rag_results['result']['data_array'], columns=["name", "description", "amenities"])
else:
    rag_df = pd.DataFrame()

## 3. Modular Agent Tools: SQL, RAG, LLM, Logging

This section defines agent tools for:
- Lodging/business search (SQL or RAG)
- Accessibility review summarization (LLM)
- Message drafting (LLM)
- MLflow logging

The agent can use either SQL (Delta tables), RAG (vector search), or demo data as needed.

In [ ]:
import mlflow
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_databricks import ChatDatabricks
from databricks.sdk import WorkspaceClient
import os

w = WorkspaceClient()
os.environ["DATABRICKS_HOST"] = w.config.host
os.environ["DATABRICKS_TOKEN"] = w.tokens.create(comment="for model serving", lifetime_seconds=1200).token_value
llm = ChatDatabricks(endpoint="databricks-llama-4-maverick", max_tokens=1024)
mlflow.langchain.autolog()

def find_accessible_lodging(city, need=None, top_n=3, use_rag=False):
    if use_rag:
        # Use RAG/vector search
        query = f"{need or ''} hotel in {city}"
        rag_results = index.similarity_search(
            columns=["name", "description", "amenities"],
            query_text=query,
            num_results=top_n
        )
        if rag_results and rag_results.get('result') and rag_results['result'].get('data_array'):
            df = pd.DataFrame(rag_results['result']['data_array'], columns=["name", "description", "amenities"])
            df["price"] = None  # Add price if available in your index
            df["reviews"] = "[]"  # Add reviews if available
            return df
        else:
            return pd.DataFrame()
    else:
        # Use demo or SQL data
        df = lodging_df[lodging_df["location"].str.contains(city, case=False)]
        if need:
            mask = (
                df["amenities"].str.contains(need, case=False, na=False) |
                df["description"].str.contains(need, case=False, na=False) |
                df["reviews"].apply(lambda reviews: any(need.lower() in r.lower() for r in reviews))
            )
            df = df[mask]
        return df.head(top_n)

def summarize_reviews_llm(reviews, need=None):
    prompt = PromptTemplate.from_template(
        """
        You are an expert travel assistant. Summarize the following user reviews for accessibility features{need_clause}.
        Reviews:
        {reviews}
        """.strip()
    )
    need_clause = f" (focus on '{need}')" if need else ""
    context = {"reviews": "\n".join(reviews), "need_clause": need_clause}
    chain = prompt | llm | StrOutputParser()
    return chain.invoke(context)

def find_accessible_businesses(city, need=None, top_n=2):
    df = business_df[business_df["location"].str.contains(city, case=False)]
    if need:
        mask = (
            df["accessibility"].str.contains(need, case=False, na=False) |
            df["reviews"].apply(lambda reviews: any(need.lower() in r.lower() for r in reviews))
        )
        df = df[mask]
    return df.sort_values("rating", ascending=False).head(top_n)

def draft_inquiry_message(place_name, need):
    prompt = PromptTemplate.from_template(
        """
        Draft a polite message to {place_name} asking to confirm if they have {need} available for guests.
        """.strip()
    )
    chain = prompt | llm | StrOutputParser()
    return chain.invoke({"place_name": place_name, "need": need})

def log_agent_action(user_query, params, results):
    mlflow.log_params(params)
    mlflow.log_text(str(results), "results.txt")
    mlflow.log_text(user_query, "user_query.txt")

## 4. Streamlit UI: Unified Experience

A simple UI for city/needs input, results display, and agent Q&A. Toggle between RAG and SQL/demo search.

In [ ]:
import streamlit as st

st.set_page_config(page_title="AccessiTrip AI", layout="wide")
st.title("AccessiTrip AI: Smart Accessible Travel Planner")

with st.sidebar:
    st.header("Plan Your Accessible Trip")
    city = st.text_input("Where are you traveling?", value="Chicago")
    need = st.text_input("Any accessibility or preference needs? (e.g. wheelchair, pet, quiet)", value="wheelchair")
    use_rag = st.checkbox("Use RAG/Vector Search", value=False)
    if st.button("Search"):
        st.session_state["search"] = True

if st.session_state.get("search"):
    # Lodging results
    st.subheader("Accessible Stays")
    stays = find_accessible_lodging(city, need, use_rag=use_rag)
    for _, row in stays.iterrows():
        st.markdown(f"**{row.get('name','')}** — ${row.get('price','N/A')} per night")
        st.markdown(f"*Amenities*: {row.get('amenities','')}")
        summary = summarize_reviews_llm(row.get("reviews", []), need) if row.get("reviews") else ""
        st.markdown(f"*Why we suggest this*: {summary}")
        st.markdown("---")

    # Businesses
    st.subheader("Nearby Accessible Restaurants/Attractions")
    businesses = find_accessible_businesses(city, need)
    for _, row in businesses.iterrows():
        st.markdown(f"**{row['name']}** ({row['type']}, {row['rating']}⭐)")
        st.markdown(f"*Accessibility*: {row['accessibility']}")
        summary = summarize_reviews_llm(row["reviews"], need)
        st.markdown(f"*Review summary*: {summary}")
        st.markdown("---")

    # Suggested inquiry message
    st.subheader("Copy Suggested Inquiry Email/Message")
    if not stays.empty:
        msg = draft_inquiry_message(stays.iloc[0]["name"], need)
        st.code(msg, language="text")

    # Log actions
    log_agent_action(
        user_query=f"city={city}, need={need}, use_rag={use_rag}",
        params={"city": city, "need": need, "use_rag": use_rag},
        results={"stays": stays.to_dict(), "businesses": businesses.to_dict()}
    )

    # Agent Q&A
    st.subheader("Not sure? Ask the Agent!")
    user_q = st.text_input("Ask a specific question (e.g. 'Is there an accessible bathroom?')")
    if st.button("Ask Agent"):
        all_reviews = []
        for _, row in stays.iterrows():
            if row.get("reviews"):
                all_reviews.extend(row["reviews"])
        answer = summarize_reviews_llm(all_reviews, user_q)
        st.markdown(f"**Agent:** {answer}")

---
## 5. Extensibility

- Each section is a modular agent tool/function.
- Swap in real Delta/Unity tables for production.
- Add more datasets (insurance, maps, etc.) as needed.
- MLflow logs all actions for traceability.

---

## 6. Demo Output Example

> **Top Accessible Hotels:**  
> - [The Grand Loop Hotel]: “Step-free entrance, elevator, user review: ‘Ramps are well maintained.’”  
> - [Wabash Suites]: “Roll-in shower, wide hallways, recent guest: ‘No stairs, great for my wheelchair!’”  
>
> **Nearby Accessible Restaurants:**  
> - [Fresh & Easy Diner]: “No steps, accessible bathroom, all employees are trained.”  
> - [Lakeside Grill]: “Accessible by public transit, quiet, braille menus.”
>
> **Suggested message to hotel:**  
> - “Hello, can you confirm your entrance and elevator accommodate wheelchairs? Are there accessible bathrooms in the guest rooms?”

---

**Ready to demo, extend, and impress!**

## 7. Code Assistant: In-Notebook and Streamlit Chat

This section enables a code assistant using Databricks LLMs. You can ask coding/data questions and get answers or code suggestions directly in the notebook or via the Streamlit UI.

In [ ]:
from langchain_databricks import ChatDatabricks
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import streamlit as st

# Instantiate a chat LLM for code assistance
code_llm = ChatDatabricks(endpoint="databricks-llama-4-maverick", max_tokens=1024)

# Define a prompt template for code Q&A
code_prompt = ChatPromptTemplate.from_template(
    """
    You are a helpful Databricks code assistant. Answer the user's coding or data question. If code is needed, provide a concise, working code snippet for Databricks (PySpark, SQL, or Python as appropriate). If the question is about data access, reference Unity Catalog/Delta best practices. If the question is about RAG or agents, reference Databricks Vector Search and LangChain integration.
    
    User question: {question}
    """
)

code_chain = code_prompt | code_llm | StrOutputParser()

def code_assistant_ask(question):
    """Ask the code assistant a question and get an answer/code snippet."""
    return code_chain.invoke({"question": question})

# Streamlit UI for code assistant
if 'code_chat_history' not in st.session_state:
    st.session_state['code_chat_history'] = []

with st.expander("💬 Code Assistant Chat (Databricks)"):
    st.markdown("Ask any Databricks coding/data question. Get code or best practices instantly!")
    code_user_q = st.text_input("Your coding/data question", key="code_assistant_input")
    if st.button("Ask Code Assistant"):
        if code_user_q:
            code_answer = code_assistant_ask(code_user_q)
            st.session_state['code_chat_history'].append((code_user_q, code_answer))
    for q, a in st.session_state['code_chat_history']:
        st.markdown(f"**You:** {q}")
        st.markdown(f"**Assistant:** {a}")

### Usage
- In notebook: Call `code_assistant_ask("your question")` in a code cell to get a code/data answer.
- In Streamlit: Use the "Code Assistant Chat" expander to ask questions and see answers interactively.
- Example: `code_assistant_ask("How do I join two Delta tables in PySpark?")`
- The assistant will return a concise answer or code snippet, referencing Databricks best practices.